<a href="https://colab.research.google.com/github/DorShabat/Cryptology-Project/blob/main/Schnorr_signature.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Schnorr signature

by https://www.youtube.com/watch?v=mV9hXEFUB6A&ab_channel=Theoretically

## pips:

In [ ]:
%%capture
!pip install cryptography

##imports:

In [ ]:
import numpy as np
import sympy
import os
import random
import hashlib
from sympy.ntheory.generate import randprime

## functions for generating the mathematical parameters:

In [ ]:
def get_biggest_factor_of_prime_minus_one(prime):
    if not sympy.isprime(prime):
        raise ValueError("The input number is not a prime number.")
    p_minus_1 = prime - 1
    factors = sympy.factorint(p_minus_1)
    return max(factors.keys())

def mod_exp(base, exp, mod):
    result = 1
    base = base % mod
    while exp > 0:
        if exp % 2 == 1:
            result = (result * base) % mod
        exp = exp >> 1
        base = (base * base) % mod
    return result

def find_a(Q, P):
    if Q == 0:
        raise ValueError("Q must be non-zero.")
    if P <= 1:
        raise ValueError("P must be greater than 1.")
    A = 2
    while A < P:
        if mod_exp(A, Q, P) == 1:
            return A
        A += 1
    raise ValueError("No non-trivial solution found.")

def mod_inverse(a, Q):
    """
    Helper function to find the modular inverse of a under modulo Q
    """
    g, x, y = extended_gcd(a, Q)
    if g != 1:
        raise ValueError(f"Modular inverse does not exist for a={a} and Q={Q}")
    else:
        return x % Q

def extended_gcd(a, b):
    """
    Helper function that implements the Extended Euclidean Algorithm
    Returns a tuple of (g, x, y) such that a*x + b*y = g = gcd(a, b)
    """
    if a == 0:
        return b, 0, 1
    else:
        g, x1, y1 = extended_gcd(b % a, a)
        x = y1 - (b // a) * x1
        y = x1
        return g, x, y

def calculate_V(a, s, Q):
    # Calculate the modular inverse of a mod Q
    a_inv = mod_inverse(a, Q)
    # Compute a^(-s) mod Q
    V = pow(a_inv, s, Q)
    return V

def hash_function(data):
    if isinstance(data, str):
        data = data.encode('utf-8')
    sha256 = hashlib.sha256()
    sha256.update(data)
    return sha256.hexdigest()

##Networking:

# Alice

***Sign:***

`M = message to be signed`

global:
```
P = a prime number     # typically 1024-bit
Q = a factor of P-1    # typically 160-bit
A = a^Q === 1 mod P
```
private key:
`s = random 0 < s < Q`

public key:
`V = a^(-s) mod Q`

for signing:
```
r = random 0 < r < Q
x = A^r mod P
e = Hash(M||x)
y = (r+se)modQ
```

Send: ` Message, M | Signature(e , y) `


### generate parameters:

In [ ]:
M = b'This is a test message with my credit card digits.' # binary msg

P = randprime(100, 200) # rand a prime number.
Q = get_biggest_factor_of_prime_minus_one(P)
A = find_a(Q, P)
s = random.randint(0, Q)
V = calculate_V(A, s, Q)
r = random.randint(0, Q)
x = mod_exp(A, r, P)
e = hash_function(str(M) + str(x))
y = (r + s*int(e, 16)) % Q

### hard coded:

In [ ]:
M = b'This is a test message with my credit card digits.' # binary msg

P = 191
Q = 19
A = 5
s = 18
V = 1
r = 2
x = 25
e = hash_function(str(M) + str(x))
y = 2


##sending to Bob......

### parameters printing:

In [ ]:
print(f'P = {P}')
print(f'Q = {Q}')
print(f'A = {A}')
print(f's = {s}')
print(f'V = {V}')
print(f'r = {r}')
print(f'x = {x}')
print(f'e = {e}')
print(f'y = {y}')

P = 163
Q = 3
A = 58
s = 0
V = 1
r = 2
x = 104
e = e36e645a1d61956a29f8af6a628f6da9d1410c44c0e6a9e8f8bd76069f6f8e04
y = 2


# Bob


*** Verification:***
  
Received:
`(M,  e,  y)`

Known publicly:
`(A,  P,  Q,  V)`

compute: `x' = (A^y * V^e) mod P`

### print what recived and known:

In [ ]:
print("   Recived form alice:\n")
print(f'M = {M}')
print(f'e = {e}')
print(f'y = {y}')
print("\n\n  known public parameters:\n")
print(f'A = {A}')
print(f'P = {P}')
print(f'Q = {Q}')
print(f'V = {V}')

   Recived form alice:

M = b'This is a test message with my credit card digits.'
e = e36e645a1d61956a29f8af6a628f6da9d1410c44c0e6a9e8f8bd76069f6f8e04
y = 2


  known public parameters:

A = 58
P = 163
Q = 3
V = 1


### compute:

In [ ]:
## recived msg form alice...

Ay = mod_exp(A, y, P)
Ve = mod_exp(V, int(e, 16), P)
AyVe = (Ay * Ve) % P
x_tag = AyVe

print(f'x\'= {x_tag}')
print(f'x = {x}\n')

e_tag = hash_function(str(M) + str(x_tag))
print(f'e\'= {e_tag}')
print(f'e = {e}')


x'= 104
x = 104

e'= e36e645a1d61956a29f8af6a628f6da9d1410c44c0e6a9e8f8bd76069f6f8e04
e = e36e645a1d61956a29f8af6a628f6da9d1410c44c0e6a9e8f8bd76069f6f8e04
